In [1]:
import os
import time
import torch
import torch as tc
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import numpy as np

In [2]:
import models
from models import weights_init

# Main code

In [3]:
#Libera as funcionalidades da biblioteca cudnn
cudnn.benchmark = True

use_gpu = torch.cuda.is_available()
#use_gpu = False
if use_gpu:
    print("You are using CUDA. If it is not what you want, manually set this as False!")

You are using CUDA. If it is not what you want, manually set this as False!


In [4]:
nc = 3
ngpu = 1
nz = 100
ngf = 64
ndf = 64
n_extra_d = 0
n_extra_g = 1 # Aqui a jogada é que o gerador deve ser mais poderoso q o detetive
imageSize = 64

In [5]:
!cd images && pwd


/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/images


## Setando as transformações

In [6]:
!ls images/images2/

ls: cannot access 'images/images2/': No such file or directory


In [7]:
!ls dataset

anime-faces


In [8]:
dataset = dset.ImageFolder(
    root='/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/dataset/anime-faces',
    transform=transforms.Compose([
            transforms.Scale((imageSize, imageSize)),
            # transforms.CenterCrop(opt.imageSize),
            transforms.ToTensor(),
            #transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)), # bring images to (-1,1)
        ])
)

## Setando o Dataloader

In [9]:
batch_size=64
num_workers = 1

In [33]:
dataloader = tc.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
#126 classes

## Setando o modelo

### Modelo DCGAN

In [11]:
class _netD_1(nn.Module):
    def __init__(self, ngpu, nz, nc, ndf,  n_extra_layers_d):
        super(_netD_1, self).__init__()
        self.ngpu = ngpu
        main = nn.Sequential(
            # input is (nc) x 96 x 96
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False), # 5,3,1 for 96x96
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
        )

        # Extra layers
        for t in range(n_extra_layers_d):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, ndf * 8),
                            nn.Conv2d(ndf * 8, ndf * 8, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, ndf * 8),
                            nn.BatchNorm2d(ndf * 8))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, ndf * 8),
                            nn.LeakyReLU(0.2, inplace=True))


        main.add_module('final_layers.conv', nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False))
        main.add_module('final_layers.sigmoid', nn.Sigmoid())
        # state size. 1 x 1 x 1
        self.main = main

    def forward(self, input):
        gpu_ids = None
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            gpu_ids = range(self.ngpu)
        output = nn.parallel.data_parallel(self.main, input, gpu_ids)
        return output.view(-1, 1)


In [12]:
# DCGAN model, fully convolutional architecture
class _netG_1(nn.Module):
    def __init__(self, ngpu, nz, nc , ngf, n_extra_layers_g):
        super(_netG_1, self).__init__()
        self.ngpu = ngpu
        #self.nz = nz
        #self.nc = nc
        #self.ngf = ngf
        main = nn.Sequential(
            # input is Z, going into a convolution
            # state size. nz x 1 x 1
            nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf) x 32 x 32
        )

        # Extra layers
        for t in range(n_extra_layers_g):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, ngf),
                            nn.Conv2d(ngf, ngf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, ngf),
                            nn.BatchNorm2d(ngf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, ngf),
                            nn.LeakyReLU(0.2, inplace=True))

        main.add_module('final_layer.deconv', 
        	             nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False)) # 5,3,1 for 96x96
        main.add_module('final_layer.tanh', 
        	             nn.Tanh())
            # state size. (nc) x 96 x 96

        self.main = main


    def forward(self, input):
        gpu_ids = None
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            gpu_ids = range(self.ngpu)
        return nn.parallel.data_parallel(self.main, input, gpu_ids), 0

In [13]:
netG = _netG_1(ngpu, nz, nc, ngf, n_extra_g)


In [14]:
print(netG)

_netG_1 (
  (main): Sequential (
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.2, inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): LeakyReLU (0.2, inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): LeakyReLU (0.2, inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): LeakyReLU (0.2, inplace)
    (extra-layers-0.64.conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (extra-layers-0.64.batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  

In [15]:
netD = _netD_1(ngpu, nz, nc, ndf, n_extra_d)

In [16]:
print(netD)

_netD_1 (
  (main): Sequential (
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU (0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU (0.2, inplace)
    (final_layers.conv): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (final_layers.sigmoid): Sigmoid ()
  )
)


# Carregando pesos pré-treinados

In [17]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


In [18]:
#Parece ser um inicializador de pesos hardcoded
netG.apply(weights_init)
netD.apply(weights_init)
print(())

()


In [19]:
load=False
if load:
    netD.load_state_dict(tc.load('path_d'))
    netG.load_state_dict(torch.load('path_G'))


### Parametros de treinamento

In [20]:
criterion = nn.BCELoss()
criterion_MSE = nn.MSELoss()


In [21]:

input = torch.FloatTensor(batch_size, 3, imageSize, imageSize)
print(input.size())
noise = torch.FloatTensor(batch_size, nz, 1, 1)
print(noise.size())

torch.Size([64, 3, 64, 64])
torch.Size([64, 100, 1, 1])


In [22]:
#parser.add_argument('--binary', action='store_true', help='z from bernoulli distribution, with prob=0.5')
binary=False
#Ele testa pergunta se vc quer que o seu Z venha da distribuição bernoulli
if binary:
    bernoulli_prob = torch.FloatTensor(batch_size, nz, 1, 1).fill_(0.5)
    fixed_noise = torch.bernoulli(bernoulli_prob)
else:
    fixed_noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)


In [23]:
label = torch.FloatTensor(batch_size)
real_label = 1
fake_label = 0


### Broadcast para CUDA, se quiser

In [24]:
if use_gpu:
    netD.cuda()
    netG.cuda()
    criterion = criterion.cuda()
    criterion_MSE = criterion_MSE.cuda()
    input,label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

### Transformando tudo em variable

In [25]:

input = Variable(input)
label = Variable(label)
noise = Variable(noise)
fixed_noise = Variable(fixed_noise)


### Setando o optimizer

In [26]:
beta1, beta2 = 0.9,0.999
lr = 2.0e-4
optimizerD = optim.Adam(netD.parameters(), lr = lr, betas = (beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr = lr, betas = (beta1, beta2))
outputDir = 'outputdir'

## Treinando !

In [27]:
print(type(input))
print(input.size())
print(label.size())

<class 'torch.autograd.variable.Variable'>
torch.Size([64, 3, 64, 64])
torch.Size([64])


In [30]:
def train_both_networks(num_epochs, dataloader, netD, netG, d_labelSmooth, outputDir, model_option =1,binary = False):
    use_gpu = tc.cuda.is_available()

    for epoch in range(num_epochs):
        for i, data in enumerate(dataloader, 0):
            start_iter = time.time()
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            # train with real
            netD.zero_grad()
            real_cpu, _ = data
            batch_size = real_cpu.size(0)
            input.data.resize_(real_cpu.size()).copy_(real_cpu)
            label.data.resize_(batch_size).fill_(real_label - d_labelSmooth) # use smooth label for discriminator

            output = netD(input)
            errD_real = criterion(output, label)
            errD_real.backward()
            D_x = output.data.mean()
            # train with fake
            noise.data.resize_(batch_size, nz, 1, 1)
            if binary:
                bernoulli_prob.resize_(noise.data.size())
                noise.data.copy_(2*(torch.bernoulli(bernoulli_prob)-0.5))
            else:
                noise.data.normal_(0, 1)
            fake,z_prediction = netG(noise)
            label.data.fill_(fake_label)
            output = netD(fake.detach()) # add ".detach()" to avoid backprop through G
            errD_fake = criterion(output, label)
            errD_fake.backward() # gradients for fake/real will be accumulated
            D_G_z1 = output.data.mean()
            errD = errD_real + errD_fake
            optimizerD.step() # .step() can be called once the gradients are computed

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            netG.zero_grad()
            label.data.fill_(real_label) # fake labels are real for generator cost
            output = netD(fake)
            errG = criterion(output, label)
            errG.backward(retain_variables=True) # True if backward through the graph for the second time
            if model_option == 2: # with z predictor
                errG_z = criterion_MSE(z_prediction, noise)
                errG_z.backward()
            D_G_z2 = output.data.mean()
            optimizerG.step()

            end_iter = time.time()
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f Elapsed %.2f s'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2, end_iter-start_iter))
            if i % 100 == 0:
                # the first 64 samples from the mini-batch are saved.
                vutils.save_image(real_cpu[0:64,:,:,:],
                        '%s/real_samples.png' % outputDir, nrow=8)
                fake,_ = netG(fixed_noise)
                vutils.save_image(fake.data[0:64,:,:,:],
                        '%s/fake_samples_epoch_%03d.png' % (outputDir, epoch), nrow=8)
        if epoch % 1 == 0:
            # do checkpointing
            torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outputDir, epoch))
            torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outputDir, epoch))

In [ ]:
num_epochs = 100
d_labelSmooth = 0.2

train_both_networks(num_epochs, dataloader, netD,netG,d_labelSmooth, outputDir)

/home/gabriel/anaconda3/envs/py35/lib/python3.5/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/gabriel/anaconda3/envs/py35/lib/python3.5/site-packages/torch/autograd/__init__.py:92: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


[0/100][0/1799] Loss_D: 0.5554 Loss_G: 4.0623 D(x): 0.7366 D(G(z)): 0.0162 / 0.0232 Elapsed 0.13 s
[0/100][1/1799] Loss_D: 0.5476 Loss_G: 3.8559 D(x): 0.7677 D(G(z)): 0.0224 / 0.0289 Elapsed 0.13 s
[0/100][2/1799] Loss_D: 0.6829 Loss_G: 4.0083 D(x): 0.9434 D(G(z)): 0.0391 / 0.0235 Elapsed 0.12 s
[0/100][3/1799] Loss_D: 0.5598 Loss_G: 4.5988 D(x): 0.8437 D(G(z)): 0.0296 / 0.0129 Elapsed 0.12 s
[0/100][4/1799] Loss_D: 0.5469 Loss_G: 4.9510 D(x): 0.7313 D(G(z)): 0.0164 / 0.0093 Elapsed 0.13 s
[0/100][5/1799] Loss_D: 0.5242 Loss_G: 5.6488 D(x): 0.8062 D(G(z)): 0.0079 / 0.0044 Elapsed 0.12 s
[0/100][6/1799] Loss_D: 0.6227 Loss_G: 5.1201 D(x): 0.6139 D(G(z)): 0.0053 / 0.0075 Elapsed 0.13 s
[0/100][7/1799] Loss_D: 0.5617 Loss_G: 4.3492 D(x): 0.7298 D(G(z)): 0.0116 / 0.0182 Elapsed 0.13 s
[0/100][8/1799] Loss_D: 0.5941 Loss_G: 3.9323 D(x): 0.8979 D(G(z)): 0.0259 / 0.0239 Elapsed 0.12 s
[0/100][9/1799] Loss_D: 0.5830 Loss_G: 4.7867 D(x): 0.8959 D(G(z)): 0.0170 / 0.0103 Elapsed 0.13 s
[0/100][10

[0/100][83/1799] Loss_D: 0.6247 Loss_G: 2.8054 D(x): 0.8959 D(G(z)): 0.0508 / 0.0723 Elapsed 0.13 s
[0/100][84/1799] Loss_D: 0.7522 Loss_G: 3.2889 D(x): 0.9512 D(G(z)): 0.0680 / 0.0449 Elapsed 0.12 s
[0/100][85/1799] Loss_D: 0.7134 Loss_G: 3.9257 D(x): 0.9414 D(G(z)): 0.0706 / 0.0230 Elapsed 0.13 s
[0/100][86/1799] Loss_D: 0.6039 Loss_G: 4.8354 D(x): 0.8830 D(G(z)): 0.0498 / 0.0113 Elapsed 0.13 s
[0/100][87/1799] Loss_D: 0.5595 Loss_G: 6.1765 D(x): 0.7060 D(G(z)): 0.0082 / 0.0029 Elapsed 0.13 s
[0/100][88/1799] Loss_D: 0.6339 Loss_G: 5.7496 D(x): 0.6038 D(G(z)): 0.0057 / 0.0044 Elapsed 0.13 s
[0/100][89/1799] Loss_D: 0.8853 Loss_G: 4.2740 D(x): 0.4070 D(G(z)): 0.0065 / 0.0187 Elapsed 0.11 s
[0/100][90/1799] Loss_D: 0.5636 Loss_G: 3.2628 D(x): 0.7405 D(G(z)): 0.0208 / 0.0528 Elapsed 0.07 s
[0/100][91/1799] Loss_D: 0.5860 Loss_G: 2.7670 D(x): 0.8251 D(G(z)): 0.0531 / 0.0822 Elapsed 0.13 s
[0/100][92/1799] Loss_D: 0.7418 Loss_G: 2.8971 D(x): 0.9254 D(G(z)): 0.1190 / 0.0714 Elapsed 0.12 s


[0/100][165/1799] Loss_D: 0.5764 Loss_G: 4.3991 D(x): 0.7069 D(G(z)): 0.0170 / 0.0167 Elapsed 0.12 s
[0/100][166/1799] Loss_D: 0.5421 Loss_G: 4.7129 D(x): 0.8079 D(G(z)): 0.0134 / 0.0121 Elapsed 0.13 s
[0/100][167/1799] Loss_D: 0.5561 Loss_G: 4.0811 D(x): 0.7592 D(G(z)): 0.0246 / 0.0234 Elapsed 0.12 s
[0/100][168/1799] Loss_D: 0.5376 Loss_G: 4.5710 D(x): 0.8020 D(G(z)): 0.0170 / 0.0151 Elapsed 0.12 s
[0/100][169/1799] Loss_D: 0.5692 Loss_G: 4.4449 D(x): 0.8725 D(G(z)): 0.0280 / 0.0169 Elapsed 0.13 s
[0/100][170/1799] Loss_D: 0.5424 Loss_G: 5.0613 D(x): 0.7789 D(G(z)): 0.0165 / 0.0104 Elapsed 0.13 s
[0/100][171/1799] Loss_D: 0.5843 Loss_G: 4.7783 D(x): 0.6745 D(G(z)): 0.0129 / 0.0129 Elapsed 0.12 s
[0/100][172/1799] Loss_D: 0.5351 Loss_G: 4.4976 D(x): 0.7713 D(G(z)): 0.0143 / 0.0150 Elapsed 0.12 s
[0/100][173/1799] Loss_D: 0.5704 Loss_G: 3.9511 D(x): 0.6975 D(G(z)): 0.0162 / 0.0239 Elapsed 0.13 s
[0/100][174/1799] Loss_D: 0.5519 Loss_G: 3.7086 D(x): 0.8351 D(G(z)): 0.0286 / 0.0302 Elaps

[0/100][247/1799] Loss_D: 0.5774 Loss_G: 5.3837 D(x): 0.6755 D(G(z)): 0.0058 / 0.0055 Elapsed 0.13 s
[0/100][248/1799] Loss_D: 0.5543 Loss_G: 4.3574 D(x): 0.7105 D(G(z)): 0.0131 / 0.0163 Elapsed 0.13 s
[0/100][249/1799] Loss_D: 0.5442 Loss_G: 4.1752 D(x): 0.7596 D(G(z)): 0.0156 / 0.0200 Elapsed 0.13 s
[0/100][250/1799] Loss_D: 0.5651 Loss_G: 4.3829 D(x): 0.8582 D(G(z)): 0.0169 / 0.0155 Elapsed 0.10 s
[0/100][251/1799] Loss_D: 0.5591 Loss_G: 4.5594 D(x): 0.8713 D(G(z)): 0.0221 / 0.0144 Elapsed 0.09 s
[0/100][252/1799] Loss_D: 0.5534 Loss_G: 4.7492 D(x): 0.7902 D(G(z)): 0.0184 / 0.0113 Elapsed 0.06 s
[0/100][253/1799] Loss_D: 0.5544 Loss_G: 4.4846 D(x): 0.7635 D(G(z)): 0.0226 / 0.0152 Elapsed 0.07 s
[0/100][254/1799] Loss_D: 0.5736 Loss_G: 4.3340 D(x): 0.6983 D(G(z)): 0.0176 / 0.0168 Elapsed 0.12 s
[0/100][255/1799] Loss_D: 0.5386 Loss_G: 4.9288 D(x): 0.8504 D(G(z)): 0.0133 / 0.0100 Elapsed 0.13 s
[0/100][256/1799] Loss_D: 0.5517 Loss_G: 4.5794 D(x): 0.7333 D(G(z)): 0.0146 / 0.0130 Elaps

...

[3/100][401/1799] Loss_D: 0.5671 Loss_G: 4.6441 D(x): 0.6941 D(G(z)): 0.0118 / 0.0125 Elapsed 0.13 s
[3/100][402/1799] Loss_D: 0.5482 Loss_G: 4.4131 D(x): 0.7239 D(G(z)): 0.0117 / 0.0162 Elapsed 0.13 s
[3/100][403/1799] Loss_D: 0.5767 Loss_G: 4.0736 D(x): 0.8789 D(G(z)): 0.0286 / 0.0247 Elapsed 0.13 s
[3/100][404/1799] Loss_D: 0.5815 Loss_G: 4.5380 D(x): 0.8795 D(G(z)): 0.0270 / 0.0145 Elapsed 0.13 s
[3/100][405/1799] Loss_D: 0.5449 Loss_G: 4.6125 D(x): 0.8083 D(G(z)): 0.0267 / 0.0130 Elapsed 0.12 s
[3/100][406/1799] Loss_D: 0.5314 Loss_G: 5.4136 D(x): 0.7645 D(G(z)): 0.0106 / 0.0060 Elapsed 0.13 s
[3/100][407/1799] Loss_D: 0.6203 Loss_G: 4.2641 D(x): 0.6201 D(G(z)): 0.0147 / 0.0172 Elapsed 0.12 s
[3/100][408/1799] Loss_D: 0.5487 Loss_G: 4.6077 D(x): 0.8350 D(G(z)): 0.0139 / 0.0129 Elapsed 0.12 s
[3/100][409/1799] Loss_D: 0.5433 Loss_G: 4.3083 D(x): 0.8022 D(G(z)): 0.0204 / 0.0168 Elapsed 0.13 s
[3/100][410/1799] Loss_D: 0.5636 Loss_G: 4.1600 D(x): 0.7554 D(G(z)): 0.0235 / 0.0197 Elaps

[3/100][483/1799] Loss_D: 0.5563 Loss_G: 5.2448 D(x): 0.7068 D(G(z)): 0.0081 / 0.0069 Elapsed 0.13 s
[3/100][484/1799] Loss_D: 0.5643 Loss_G: 4.4544 D(x): 0.7039 D(G(z)): 0.0121 / 0.0148 Elapsed 0.13 s
[3/100][485/1799] Loss_D: 0.5340 Loss_G: 4.4505 D(x): 0.7584 D(G(z)): 0.0108 / 0.0144 Elapsed 0.13 s
[3/100][486/1799] Loss_D: 0.5557 Loss_G: 4.2920 D(x): 0.8670 D(G(z)): 0.0206 / 0.0176 Elapsed 0.13 s
[3/100][487/1799] Loss_D: 0.5777 Loss_G: 4.5240 D(x): 0.8899 D(G(z)): 0.0250 / 0.0137 Elapsed 0.12 s
[3/100][488/1799] Loss_D: 0.5703 Loss_G: 5.0210 D(x): 0.8685 D(G(z)): 0.0238 / 0.0089 Elapsed 0.13 s
[3/100][489/1799] Loss_D: 0.6435 Loss_G: 4.7436 D(x): 0.5849 D(G(z)): 0.0092 / 0.0113 Elapsed 0.12 s
[3/100][490/1799] Loss_D: 0.5443 Loss_G: 4.6498 D(x): 0.7147 D(G(z)): 0.0082 / 0.0120 Elapsed 0.12 s
[3/100][491/1799] Loss_D: 0.5565 Loss_G: 3.6350 D(x): 0.7209 D(G(z)): 0.0201 / 0.0309 Elapsed 0.12 s
[3/100][492/1799] Loss_D: 0.5558 Loss_G: 3.7355 D(x): 0.8442 D(G(z)): 0.0266 / 0.0285 Elaps

[3/100][565/1799] Loss_D: 0.5476 Loss_G: 4.9140 D(x): 0.7295 D(G(z)): 0.0080 / 0.0096 Elapsed 0.13 s
[3/100][566/1799] Loss_D: 0.5490 Loss_G: 3.9829 D(x): 0.7396 D(G(z)): 0.0179 / 0.0236 Elapsed 0.13 s
[3/100][567/1799] Loss_D: 0.5631 Loss_G: 4.0654 D(x): 0.8581 D(G(z)): 0.0251 / 0.0228 Elapsed 0.13 s
[3/100][568/1799] Loss_D: 0.5944 Loss_G: 3.8533 D(x): 0.8695 D(G(z)): 0.0490 / 0.0262 Elapsed 0.13 s
[3/100][569/1799] Loss_D: 0.5360 Loss_G: 5.3442 D(x): 0.7816 D(G(z)): 0.0116 / 0.0067 Elapsed 0.12 s
[3/100][570/1799] Loss_D: 0.5352 Loss_G: 5.0782 D(x): 0.7840 D(G(z)): 0.0141 / 0.0080 Elapsed 0.13 s
[3/100][571/1799] Loss_D: 0.5398 Loss_G: 5.3533 D(x): 0.8233 D(G(z)): 0.0125 / 0.0065 Elapsed 0.12 s
[3/100][572/1799] Loss_D: 0.6064 Loss_G: 5.3873 D(x): 0.6284 D(G(z)): 0.0055 / 0.0058 Elapsed 0.13 s
[3/100][573/1799] Loss_D: 0.5759 Loss_G: 4.4382 D(x): 0.6547 D(G(z)): 0.0092 / 0.0160 Elapsed 0.13 s
[3/100][574/1799] Loss_D: 0.5807 Loss_G: 4.0353 D(x): 0.8968 D(G(z)): 0.0219 / 0.0218 Elaps

[3/100][647/1799] Loss_D: 0.5787 Loss_G: 4.8951 D(x): 0.6620 D(G(z)): 0.0123 / 0.0098 Elapsed 0.13 s
[3/100][648/1799] Loss_D: 0.5460 Loss_G: 4.8087 D(x): 0.8399 D(G(z)): 0.0159 / 0.0102 Elapsed 0.13 s
[3/100][649/1799] Loss_D: 0.5327 Loss_G: 5.8437 D(x): 0.8449 D(G(z)): 0.0073 / 0.0039 Elapsed 0.13 s
[3/100][650/1799] Loss_D: 0.5416 Loss_G: 5.0420 D(x): 0.7371 D(G(z)): 0.0120 / 0.0088 Elapsed 0.13 s
[3/100][651/1799] Loss_D: 0.5357 Loss_G: 4.9749 D(x): 0.7501 D(G(z)): 0.0115 / 0.0099 Elapsed 0.13 s
[3/100][652/1799] Loss_D: 0.5269 Loss_G: 5.5545 D(x): 0.8139 D(G(z)): 0.0068 / 0.0056 Elapsed 0.13 s
[3/100][653/1799] Loss_D: 0.5380 Loss_G: 4.9132 D(x): 0.7304 D(G(z)): 0.0090 / 0.0094 Elapsed 0.12 s
[3/100][654/1799] Loss_D: 0.5433 Loss_G: 4.6594 D(x): 0.8481 D(G(z)): 0.0150 / 0.0121 Elapsed 0.13 s
[3/100][655/1799] Loss_D: 0.5382 Loss_G: 5.0705 D(x): 0.7787 D(G(z)): 0.0104 / 0.0089 Elapsed 0.13 s
[3/100][656/1799] Loss_D: 0.5374 Loss_G: 4.5134 D(x): 0.8093 D(G(z)): 0.0203 / 0.0149 Elaps

[3/100][729/1799] Loss_D: 0.5737 Loss_G: 3.8696 D(x): 0.8649 D(G(z)): 0.0260 / 0.0290 Elapsed 0.12 s
[3/100][730/1799] Loss_D: 0.6631 Loss_G: 4.3663 D(x): 0.9345 D(G(z)): 0.0337 / 0.0181 Elapsed 0.13 s
[3/100][731/1799] Loss_D: 0.5689 Loss_G: 4.6901 D(x): 0.8509 D(G(z)): 0.0309 / 0.0117 Elapsed 0.13 s
[3/100][732/1799] Loss_D: 0.5331 Loss_G: 5.9819 D(x): 0.7875 D(G(z)): 0.0089 / 0.0035 Elapsed 0.12 s
[3/100][733/1799] Loss_D: 0.5775 Loss_G: 5.4378 D(x): 0.6685 D(G(z)): 0.0079 / 0.0063 Elapsed 0.13 s
[3/100][734/1799] Loss_D: 0.5575 Loss_G: 5.0636 D(x): 0.7052 D(G(z)): 0.0064 / 0.0079 Elapsed 0.13 s
[3/100][735/1799] Loss_D: 0.5912 Loss_G: 4.3374 D(x): 0.6423 D(G(z)): 0.0070 / 0.0168 Elapsed 0.06 s
[3/100][736/1799] Loss_D: 0.5637 Loss_G: 4.0582 D(x): 0.8772 D(G(z)): 0.0167 / 0.0218 Elapsed 0.13 s
[3/100][737/1799] Loss_D: 0.6455 Loss_G: 3.8545 D(x): 0.9247 D(G(z)): 0.0461 / 0.0268 Elapsed 0.12 s
[3/100][738/1799] Loss_D: 0.5944 Loss_G: 4.8622 D(x): 0.8881 D(G(z)): 0.0281 / 0.0101 Elaps